# WaveMesh-Diff - Google Colab Quick Start

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/HoangNguyennnnnnn/WaveMeshDf/blob/main/colab_quickstart.ipynb)

**3D Mesh Generation using Diffusion Models in Wavelet Domain**

---

## ⚡ Quick Overview

This notebook demonstrates:
1. ✅ Setup WaveMesh-Diff in Google Colab
2. 🧪 Test all 4 modules (Wavelet, U-Net, Diffusion, Multi-view)
3. 📊 Visualize sparse wavelet representation
4. 🎨 Run quick demos

**Estimated time: 10-15 minutes**

---

## 🚀 Setup

### 1. Clone Repository

In [ ]:
!git clone https://github.com/HoangNguyennnnnnn/WaveMeshDf.git
%cd WaveMeshDf

### 2. Install Dependencies

In [ ]:
# Cài dependencies cơ bản
!pip install -q PyWavelets trimesh matplotlib rtree scipy

# PyTorch thường đã có sẵn trong Colab
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

In [ ]:
# Cài thêm scikit-image cho marching cubes
!pip install -q scikit-image

print("✅ All dependencies installed!")

In [ ]:
# Quick verification - check imports work
try:
    import pywt
    import trimesh
    import matplotlib
    from skimage import measure
    print("✅ PyWavelets:", pywt.__version__)
    print("✅ Trimesh:", trimesh.__version__)
    print("✅ Matplotlib:", matplotlib.__version__)
    print("✅ scikit-image: OK")
    print("\n🎉 All core dependencies ready!")
except ImportError as e:
    print(f"❌ Missing dependency: {e}")
    print("Run the install cells above to fix this.")

### 3. Optional: Install Advanced Features

In [ ]:
# Cài transformers cho DINOv2 (tùy chọn - cải thiện quality)
!pip install -q transformers huggingface_hub

# Login HuggingFace (cần token từ https://huggingface.co/settings/tokens)
# from huggingface_hub import login
# login(token="your_token_here")

---

## 🧪 Test Installation

In [ ]:
# Test tất cả modules
!python test_all_modules.py

**Note:** Nếu gặp lỗi import, restart runtime và chạy lại từ đầu.

**Kỳ vọng:**
```
Results: 4/4 modules passed
  Module A ✅ PASS
  Module B ✅ PASS
  Module C ✅ PASS
  Module D ✅ PASS
```

---

## 📊 Quick Demo

### Module A: Wavelet Transform

In [ ]:
from data.wavelet_utils import mesh_to_sdf_simple, sdf_to_sparse_wavelet, sparse_wavelet_to_sdf
import trimesh
import numpy as np
import matplotlib.pyplot as plt

# Tạo mesh mẫu
mesh = trimesh.creation.box(extents=[1, 1, 1])
print(f"Mesh: {len(mesh.vertices)} vertices, {len(mesh.faces)} faces")

# Chuyển sang SDF
sdf = mesh_to_sdf_simple(mesh, resolution=32)
print(f"SDF shape: {sdf.shape}")

# Wavelet transform - trả về dictionary
sparse_data = sdf_to_sparse_wavelet(sdf, threshold=0.01)
print(f"Sparse indices: {sparse_data['indices'].shape}")
print(f"Sparse features: {sparse_data['features'].shape}")

# Calculate sparsity
total_elements = 32 ** 3
non_zero = len(sparse_data['features'])
sparsity = 100 * (1 - non_zero / total_elements)
print(f"Sparsity: {sparsity:.1f}%")

# Reconstruct
sdf_recon = sparse_wavelet_to_sdf(sparse_data)
mse = np.mean((sdf - sdf_recon) ** 2)
print(f"Reconstruction MSE: {mse:.6f}")

### Visualize SDF

In [ ]:
# Visualize SDF slice
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].imshow(sdf[16, :, :], cmap='RdBu')
axes[0].set_title('Original SDF (slice)')
axes[0].axis('off')

axes[1].imshow(sdf_recon[16, :, :], cmap='RdBu')
axes[1].set_title('Reconstructed SDF')
axes[1].axis('off')

diff = np.abs(sdf - sdf_recon)
axes[2].imshow(diff[16, :, :], cmap='hot')
axes[2].set_title(f'Error (MSE={mse:.6f})')
axes[2].axis('off')

plt.tight_layout()
plt.show()

---

### Module D: Multi-view Encoder

In [ ]:
from models import create_multiview_encoder
import torch

# Tạo encoder
encoder = create_multiview_encoder(preset='small')
print(f"Encoder created: {sum(p.numel() for p in encoder.parameters()):,} params")

# Test với random data
batch_size = 2
num_views = 4
images = torch.randn(batch_size, num_views, 3, 224, 224)
poses = torch.randn(batch_size, num_views, 3, 4)

# Forward pass
with torch.no_grad():
    conditioning = encoder(images, poses)

print(f"Input images: {images.shape}")
print(f"Input poses: {poses.shape}")
print(f"Output conditioning: {conditioning.shape}")
print("✅ Multi-view encoder working!")

---

### Module B + C: U-Net + Diffusion

In [ ]:
from models import WaveMeshUNet, GaussianDiffusion

# Tạo U-Net
unet = WaveMeshUNet(
    in_channels=1,
    encoder_channels=[16, 32, 64],
    decoder_channels=[64, 32, 16],
    time_emb_dim=128,
    use_attention=True,
    context_dim=384  # Match Module D output
)
print(f"U-Net: {sum(p.numel() for p in unet.parameters()):,} params")

# Tạo Diffusion
diffusion = GaussianDiffusion(
    timesteps=1000,
    beta_schedule='linear'
)
print(f"Diffusion: {diffusion.timesteps} timesteps")
print(f"Beta range: [{diffusion.betas[0]:.6f}, {diffusion.betas[-1]:.6f}]")
print("✅ U-Net + Diffusion ready!")

---

## 📊 Download Data

### Option 1: ModelNet40 (Quick - 500MB)

In [ ]:
# Download ModelNet40
!python scripts/download_data.py --dataset modelnet40

# Check downloaded data
!ls -lh data/ModelNet40/ 2>/dev/null || echo "Data downloading... Check scripts/download_data.py for manual instructions"

### Option 2: ShapeNet (Manual)

Để download ShapeNet:
1. Đăng ký tại https://shapenet.org/
2. Download ShapeNetCore.v2
3. Upload lên Google Drive
4. Mount Drive và copy data

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Copy ShapeNet data (nếu đã có trong Drive)
# !cp -r /content/drive/MyDrive/ShapeNetCore.v2 ./data/

---

## 🎨 Visualize Pipeline

In [ ]:
# Visualize complete pipeline
# Note: visualize_results.py cần được tạo trước
# Hoặc dùng code đơn giản dưới đây:

import matplotlib.pyplot as plt
from data.wavelet_utils import WaveletTransform3D
import numpy as np

# Simple visualization
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('WaveMesh-Diff Pipeline Overview', fontsize=16, fontweight='bold')

# 1. Input SDF
axes[0, 0].imshow(sdf[16, :, :], cmap='RdBu')
axes[0, 0].set_title('1. Input SDF (slice)')
axes[0, 0].axis('off')

# 2. Wavelet Coefficients (visualize sparsity)
sparse_indices = sparse_data['indices']
sparse_viz = np.zeros((32, 32))
for idx in sparse_indices:
    if idx[2] == 16:  # Same slice
        sparse_viz[idx[0], idx[1]] += 1
axes[0, 1].imshow(sparse_viz, cmap='hot')
axes[0, 1].set_title(f'2. Sparse Wavelet ({sparsity:.1f}% sparse)')
axes[0, 1].axis('off')

# 3. Reconstructed SDF
axes[1, 0].imshow(sdf_recon[16, :, :], cmap='RdBu')
axes[1, 0].set_title('3. Reconstructed SDF')
axes[1, 0].axis('off')

# 4. Reconstruction Error
error = np.abs(sdf - sdf_recon)
axes[1, 1].imshow(error[16, :, :], cmap='Reds')
axes[1, 1].set_title(f'4. Error (MSE={mse:.6f})')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()
print("✅ Pipeline visualization complete!")

---

## 🏋️ Training Example (Conceptual)

⚠️ **Lưu ý:** Training đầy đủ cần nhiều thời gian và GPU. Xem `ROADMAP.md` để có code đầy đủ.

In [ ]:
# Training loop (conceptual)
# Xem ROADMAP.md để có implementation đầy đủ

from models import create_multiview_encoder, WaveMeshUNet, GaussianDiffusion
import torch

# 1. Prepare models
encoder = create_multiview_encoder(preset='small')
unet = WaveMeshUNet(
    in_channels=1,
    encoder_channels=[16, 32, 64],
    decoder_channels=[64, 32, 16],
    time_emb_dim=128,
    context_dim=384
)
diffusion = GaussianDiffusion(timesteps=1000)

# 2. Optimizer
optimizer = torch.optim.AdamW([
    {'params': encoder.parameters(), 'lr': 1e-5},
    {'params': unet.parameters(), 'lr': 1e-4}
])

print("✅ Training setup ready!")
print(f"Encoder params: {sum(p.numel() for p in encoder.parameters()):,}")
print(f"U-Net params: {sum(p.numel() for p in unet.parameters()):,}")
print("\n📖 See ROADMAP.md for full training code with:")
print("  • Dataset implementation")
print("  • Training loop")
print("  • Evaluation metrics")
print("  • Checkpointing")

---

## 📚 Next Steps

1. **Đọc Documentation:**
   - [README.md](README.md) - Project overview
   - [QUICKSTART.md](QUICKSTART.md) - Quick start guide
   - [ROADMAP.md](ROADMAP.md) - Training roadmap
   - [ARCHITECTURE.md](ARCHITECTURE.md) - Technical details

2. **Download Data:**
   - ModelNet40 (500MB) - Quick start
   - ShapeNet (50GB) - Better quality

3. **Train Model:**
   - Xem `ROADMAP.md` để có training code đầy đủ
   - Implement dataset loader
   - Run training loop

4. **Improve:**
   - Mixed precision training
   - Classifier-free guidance
   - EMA for better quality

---

## 🐛 Troubleshooting

### "ModuleNotFoundError: No module named 'pywt'"
```python
!pip install PyWavelets
```

### "ModuleNotFoundError: No module named 'rtree'"
```python
!pip install rtree
```

### "ValueError: too many values to unpack (expected 2)"
Lỗi này xảy ra khi dùng API cũ. `sdf_to_sparse_wavelet()` trả về **dictionary**, không phải tuple:
```python
# ❌ Sai:
coeffs, coords = sdf_to_sparse_wavelet(sdf)

# ✅ Đúng:
sparse_data = sdf_to_sparse_wavelet(sdf, threshold=0.01)
print(sparse_data['indices'].shape)
print(sparse_data['features'].shape)
```

### "CUDA out of memory"
```python
# Giảm batch size hoặc resolution
batch_size = 2
resolution = 16
```

### "transformers not available"
```python
!pip install transformers huggingface_hub
# Code sẽ tự động fallback sang CNN nếu không có
```

### "No module named 'skimage'"
```python
!pip install scikit-image
```

Xem đầy đủ tại [TROUBLESHOOTING.md](TROUBLESHOOTING.md).

---

**Happy 3D Generation! 🎨**